In [1]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [2]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set("spark.driver.maxResultSize", "5g"))
sc = pyspark.SparkContext(conf=conf)

In [3]:
!dir /users/trush/CSC496/Labs/Lab4

data  Lab4.ipynb


In [4]:
!ls -lh /users/trush/CSC496/Labs/Lab4/data

total 5.9G
-rw-r--r-- 1 trush PDC-edu-Lab  27K Nov 19 08:03 'lab-4-lngo(1).ipynb'
-rw-r--r-- 1 trush PDC-edu-Lab  25M Nov 19 07:59  yelp_academic_dataset_business.json.gz
-rw-r--r-- 1 trush PDC-edu-Lab 2.4G Nov 19 08:05  yelp_academic_dataset_review.json.gz
-rw-r--r-- 1 trush PDC-edu-Lab 3.5G Nov 19 08:04  yelp_academic_dataset_user.json.gz


# 1. Identify 100 users with highest number of ratings/reviews/followers.  You can decide/justify this ranking and decide the importance of ratings/reviews/followers. 

In [5]:
# Step 1: Load the data using SQL Context

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [6]:
# Get SQL table
df_json = sqlContext.read.json("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_user.json.gz")

Py4JJavaError: An error occurred while calling o27.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, pcvm604-2.emulab.net, executor driver): java.io.IOException: invalid stored block lengths
	at org.apache.hadoop.io.compress.zlib.BuiltInGzipDecompressor.decompress(BuiltInGzipDecompressor.java:209)
	at org.apache.hadoop.io.compress.DecompressorStream.decompress(DecompressorStream.java:111)
	at org.apache.hadoop.io.compress.DecompressorStream.read(DecompressorStream.java:105)
	at java.base/java.io.InputStream.read(InputStream.java:205)
	at org.apache.hadoop.util.LineReader.fillBuffer(LineReader.java:182)
	at org.apache.hadoop.util.LineReader.readDefaultLine(LineReader.java:218)
	at org.apache.hadoop.util.LineReader.readLine(LineReader.java:176)
	at org.apache.hadoop.mapreduce.lib.input.LineRecordReader.nextKeyValue(LineRecordReader.java:194)
	at org.apache.spark.sql.execut

In [ ]:
df_json.printSchema()

In [ ]:
df_json.registerTempTable("tbl_json")

In [ ]:
# Using Python
import json
def parseJSONReviews(x):
    res = json.loads(x)
    return (res['user_id'], int(res['review_count']), res['fans'])

In [ ]:
df_data = sc.textFile("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_user.json.gz")
print(df_data.count())
df_data.take(1)

In [ ]:
df_highest = df_data.map(lambda x: parseJSONReviews(x)).takeOrdered(100, lambda x: -x[2])
df_highest

In [ ]:
# Using SQL
highest_fan = sqlContext.sql("SELECT user_id, review_count, fans FROM tbl_json ORDER BY fans DESC LIMIT 100")

In [ ]:
highest_fan.show()

# 2. Extract the reviews of these users and combine it with the business information.

### a. Are they continental, regional, or local eaters?
- Look at review data for business data. Look at business data for location.
- Variations in latitude and longitude of reviewed businesses (distance between furthest pair of restaurants, cluster into 3)

### b. Is there a preference in restaurant/food style of their reviews?
- Frequent itemsets
    - per user
        - set of items: categories
        - set of baskets: restaurants

### c. Can you infer the locations of these users?
- Locations of businesses most frequently visited (consider timestamp for users who go on trips to review restaurants)

#### How to prepare data to answer questions?
- List of users with highest fans (user_id, review_count, fans) - local
- Map users to reviews

In [ ]:
#raw_review_df.filter(lambda x: json.loads(x)["user_id"] in df_top_users)

In [ ]:
def extract_id(x):
    x_json = json.loads(x)
    return (x_json["user_id"])
tmp = df_data.take(1)
extract_id(tmp[0])

# 3. Identify one of your favorite restaurants that is available on Yelp. Search for all reviews and reviewers for this restaurants.  

### a. Is this restaurant frequented by non-local reviewers (how do you know)?
- Coach's

### b. What are the positive things about this restaurant (study higher-rated reviews)
- sentiment analysis

### c. What are the negative things about this restaurant (study lower-rated reviews)
- sentiment analysis

In [ ]:
raw_review_df = sc.textFile("/users/trush/CSC496/Labs/Lab4/data/yelp_academic_dataset_review.json.gz")
print(raw_review_df)
raw_review_df.take(1)

In [ ]:
user_reviews = df_user_review.map(lambda x: (x[1], 1))
user_reviews.take(10)

In [ ]:
df_user_reviews = raw_review_df.filter(lambda x: json.loads(x)["user_id"] in df_data)
print(df_user_reviews.count())
df_user_reviews.take(10)

In [ ]:
df_test = raw_review_df.take(1)
json.loads(df_test)["user_id"]

In [ ]:
sum = 0
for item in df_top_users:
    sum = sum + int(item[1])
sum